# Mini projet 3: Les profils PSSM

Le but de ce projet est de construire une PSSM (Position Specific Scoring Matrix) pour représenter un motif (patterns) dans un groupe de séquence de la famille WW dans notre cas et ainsi représenter cette famille dans son ensemble .

Pour cela on doit en premier lieu implémenter l'algorithme de création d'une PSSM.

Ensuite comparer nos résultat en utilisant la même famille avec ceux qu'on peut trouver sur le site PFAM.

## Données

L'ensemble de séquence sur lequel on doit executer nos algorithmes se trouve comme cité dans l'énoncé dans la base de donnée [SMART](http://smart.embl.de) en utilisant le mode _normal_.

Plus précisément on utilise les 136 séquences qui sont liées aux domaines WW dans des protéines humaines (sauvegardé dans le fichier _to-be-aligned.fasta_).

Ces séquences peuvent être alignées à l'aide de trois outils proposé dans l'énoncé parmis lesquels j'ai choisis ces deux ci:

* [CLUSTAL Omega](http://www.ebi.ac.uk/Tools/msa/clustalo/) (alignement sauvegardé dans _msaresults-CLUSTAL.fasta_) 
* [MUSCLE](http://www.ebi.ac.uk/Tools/msa/muscle/) (alignement sauvegardé dans _msaresults-MUSCLE.fasta_) 

## Implémentation

Je réutilise le parser que j'ai utilisez dans mes précédents projets pour lire des fichiers _.fasta_ qui viennent d'être alignés.

In [17]:
VALID_AMINO = "ARNDCQEGHILKMFPSTWYVBZX"

In [36]:
class Sequence():
    """
    ADT séquence qui représente une séquence d’acides aminés
    et tous les opérations qu’on peut exécuter sur une séquence.
    """
    def __init__(self, sequence):
        self.current = 0
        self.sequence = sequence
    
    def __repr__(self):
        return str(self.sequence)
    
    def __len__(self):
        return len(self.sequence)
    
    def __iter__(self):
        self.current = 0
        return self
    
    def __next__(self):
        if self.current >= len(self.sequence):
            raise StopIteration
        else:
            self.current += 1
            return self.sequence[self.current - 1]
    
    def __getitem__(self, index):
        """
        @desc: Permet d'interpreter la classe comme une "String".
        
        @param{index}: Index de la lettre qu'on veut consulter.
        """
        return self.sequence[index]

    @staticmethod
    def load(filename):
        """
        """
        with open(filename) as f:
            seq = []
            for line in f:
                if line[0] == ">":
                    if seq:
                        yield Sequence(''.join(seq))
                    seq = [] 
                else:
                    seq.append(line.strip())
                    
            if (seq):
                yield Sequence(''.join(seq))

J'importe les résultats des alignements executé à l'aide des outils _CLUSTAL Omega_ et _MUSCLE_ dans le notebook.

In [37]:
clustal = [seq for seq in Sequence.load('./msaresults-CLUSTAL.fasta')]
muscle = [seq for seq in Sequence.load('./msaresults-MUSCLE.fasta')]

## L'algorithme

On rechercher ici à déterminer la probabilité qu'a un certain acide aminé d'apparaitre à une certaine position dans une séquence, là où les anciens projets nous demandait d'obtenir un score d'alignement.

In [38]:
# les proba sont hard-coder | trouver sur swissprot
PROBABILITY_AMINO = {
    "A": 0.0826,   "Q": 0.0393,   "L": 0.0966,   "S": 0.0657,
    "R": 0.0553,   "E": 0.0674,   "K": 0.0583,   "T": 0.0534,
    "N": 0.0405,   "G": 0.0708,   "M": 0.0241,   "W": 0.0109,
    "D": 0.0546,   "H": 0.0227,   "F": 0.0386,   "Y": 0.0292,
    "C": 0.0137,   "I": 0.0595,   "P": 0.0471,   "V": 0.0687,
    "B": 0.0000,   "Z": 0.0000,  "X": 0.0000
}

In [63]:
from math import log, sqrt


class Profil(object):
    """ 
    Classe permetant de créer un profile 
    """
    def __init__(self, sequences):
        self._sequences = sequences
        self._seqSize = len(self._sequences[0])

        self._alpha = len(self._sequences) - 1
        self._beta = sqrt(len(self._sequences))

        self._profile = [{a : 0 for a in VALID_AMINO} for i in range(self._seqSize)]

        self._consensusSeq = ""

        self._make()

    def _addFrequency(self):
        """
        Ajoute la fréquence pour chaque acide aminé au profile 
        """
        for seq in self._sequences:
            for i, amino in enumerate(seq):
                if amino != "-":
                    self._profile[i][amino] += (1 / len(self._sequences))

    def _addQ(self):
        """
        Ajoute des pseudo count au profile
        """
        for i in range(self._seqSize):
            for amino in VALID_AMINO:
                self._profile[i][amino] = (self._alpha * self._profile[i][amino]) + (self._beta * PROBABILITY_AMINO[amino])
                self._profile[i][amino] /= (self._alpha + self._beta)

    def _addScore(self):
        """ 
        Ajoute le score (log finale) au profile
        """
        for i in range(self._seqSize):
            for amino in VALID_AMINO:
                if PROBABILITY_AMINO[amino] == 0 and self._profile[i][amino] == 0:
                    # limite log tendant => 0 ==  (log(0/0)) == -inf
                    # car une proba réellement nul n'est pas possible => log(0)
                    self._profile[i][amino] = float("-inf")
                elif PROBABILITY_AMINO[amino] != 0: #log(nbr-reel)
                    self._profile[i][amino] = log(self._profile[i][amino] / PROBABILITY_AMINO[amino], 2)
                else: # log(inf)
                    self._profile[i][amino] = float("inf")

    def _makeConsensus(self):
        """ Crée la séquence consensus"""
        self._consensusSeq = ""
        for i in range(self._seqSize):
            # on ajoute a chaque fois le meilleur amino au consensus
            bestAmino = VALID_AMINO[0]
            for amino in VALID_AMINO:
                if self._profile[i][amino] > self._profile[i][bestAmino]:
                    bestAmino = amino
                    
            self._consensusSeq += bestAmino

    def _make(self):
        """
        Cette méthode ajoute les différentes étapes au profil et crée le consensus
        """
        self._addFrequency()
        self._addQ()
        self._addScore()
        
        self._makeConsensus()

    def __str__(self):
        """ 
        Cette méthode sauvegarde le profile dans un fichier
        """
        # Première ligne : numéro des colonnes
        result = "   "
        for i in range(self._seqSize):
            result += "%8i" % (i)
        result += "\n"
        
        for amino in VALID_AMINO:
            result += "%s  " % (amino)
            for i in range(self._seqSize):
                result += "%+8.3f" % (round(self._profile[i][amino], 3))
            result += "\n"
            
        return result

Le but de l'algorithme est de représenter un ensemble de protéine. Le calcul du score est effectué en plusieurs étapes.

    1. Calcul des fŕéquences d'acides aminées dans chaque colonne.
    
    2. Calcul de la probabilité de trouver un acide aminé dans une séquence.

Pour calculer la fréquence d'apparition de chaque acide aminé sur l'ensemble des séquences on doit trouver:

$$f_{u,b} = \frac{n_{u,b}}{N_{seq}}$$

qui est la fréquence d'apparition d'un acide aminé sur une séquence avec $n_{u,b}$ qui est le nombre de fois qu'apparait un acide aminé dans une séquence et $N_{seq}$ qui est le nombre de séquence.

Ce calcul est défini dans la fonction `def _addFrequency(self):`.

La deuxième étape est de calculer la probabilitée de trouver un acide aminé dans une séquence. Avec:

$$N'_{seq} = nombre\ de\ séquence\ sans\ "gap"\ à\ cette\ position$$
$$\alpha = N_{seq} - 1$$
$$\beta = \sqrt{N_{seq}}$$

$p_a$ est la probabilité d'obtenir un acide aminé (trouvé sur _SWISSPROT_) que j'ai traduit comme suit dans mon implémentation:

```
PROBABILITY_AMINO = {
    "A": 0.0826,   "Q": 0.0393,   "L": 0.0966,   "S": 0.0657,
    "R": 0.0553,   "E": 0.0674,   "K": 0.0583,   "T": 0.0534,
    "N": 0.0405,   "G": 0.0708,   "M": 0.0241,   "W": 0.0109,
    "D": 0.0546,   "H": 0.0227,   "F": 0.0386,   "Y": 0.0292,
    "C": 0.0137,   "I": 0.0595,   "P": 0.0471,   "V": 0.0687,
    "B": 0.0000,   "Z": 0.0000,  "X": 0.0000
}
```

La probabilitée est donc:

$$q_{u,a} = \frac{\alpha f_{u,a} + \beta p_a}{\alpha + \beta}$$

La probabilitée est calculée dans la méthode `def _addFrequency(self):`


Le score final est calculé avec la formule suivante:

$$m_{u,a} = log \frac{q_{u,a}}{p_a}$$

#### Note

Il est possible que la fonction $q_{u,a}$ aie un résultat égale à 0, ce qui donne une forme indéterminée. Pour cela on peut utiliser les pseudocounts qui sont des constantes qu'on ajoute aux valeurs dans le profile qui représentent le distribution antérieure. 

C'est pour cela qu'on ajoute à $\frac{\alpha f_{u,a}}{\alpha + \beta}$: $\frac{\beta p_a}{\alpha + \beta}$ dans notre calcul de $q_{u,a}$.

De cette manière même quand il n'y a aucune séquence, les _pseudocounts_ déterminent la valeurs de notre profile

# Analyse des résultats

On commence par executer les profils sur les séquences.

In [64]:
c = Profil(clustal)
m = Profil(muscle)

In [65]:
print(m)
print(m._consensusSeq)

          0       1       2       3       4       5       6       7       8       9      10      11      12      13      14      15      16      17      18      19      20      21      22      23      24      25      26      27      28      29      30      31      32      33      34      35      36      37      38      39      40      41      42      43      44      45      46      47      48      49      50      51      52      53      54      55      56      57      58      59
A    -1.571  -1.386  -2.335  -0.027  -3.236  -4.058  -4.058  -1.571  +0.702  -4.058  -0.941  -4.058  +0.163  -1.386  -4.058  -4.058  -1.075  -3.236  -4.058  -4.058  -1.386  -4.058  -4.058  -4.058  -4.058  -4.058  -3.236  -4.058  -3.236  -4.058  -4.058  -4.058  -4.058  -4.058  -4.058  -4.058  -4.058  -4.058  -4.058  -4.058  -4.058  -4.058  -4.058  -4.058  -4.058  -4.058  -4.058  -0.327  -2.033  -1.222  -4.058  -1.386  -3.236  -1.075  -4.058  -1.784  -4.058  -4.058  -1.222  -1.784
R    -4.058  -2.339  -1.576  +0.

## Weblogo sur MUSCLE

![](weblogo_MUSCLE.png)

# Weblogo sur CLUSTAL

![](weblogo_CLUSTAL.png)